In [ ]:
import import_ipynb

## Use resize when calc mean, std/ num_epoch = 5 / use transform 1/ data set num: 100 / optim: SGD/ batch: 64

Epoch [1/5], Loss: 0.6829988956451416
Epoch [2/5], Loss: 0.660178005695343
Epoch [3/5], Loss: 0.6521103978157043
Epoch [4/5], Loss: 0.6584694385528564
Epoch [5/5], Loss: 0.6707316040992737
checking accuracy on Training set
got 57 / 100 with accracy 56.99999999999999
checking accuracy on Test set
got 8 / 20 with accracy 40.0

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

import os

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 64
num_epochs = 5

In [ ]:
# transform 1

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    ToTensor()
])


In [ ]:
# transform 2
# transform = transforms.Compose([
#    transforms.Resize((224, 224)),
#    ToTensor(),
# #    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
#     transforms.Normalize(mean, std)
# ])


In [ ]:
# transform 3
# transform = transforms.Compose([
#         transforms.RandomResizedCrop(224),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize(mean, std, inplace=False)
# ])

In [ ]:

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)

In [ ]:

print("Checking dataset...")


for i in range(5):
    image, label = dataset[i]
    print(f"Image {i} shape: {image.shape}, Label: {label}")

print("\nChecking train loader...")


for images, labels in train_loader:
    print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
    break  

print("\nChecking test loader...")


for images, labels in test_loader:
    print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
    break  


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Linear(5000, 2),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits


In [ ]:
# model

model = NeuralNetwork().to(device)
print(model)

In [ ]:
# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()

In [ ]:

# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
#optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

In [ ]:
# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

    # training mode
    model.train()

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)

## Use resize when calc mean, std/ num_epoch = 10 / use transform 1/ data set num: 100 / optim: SGD/ batch: 64

Epoch [1/10], Loss: 0.688021719455719
Epoch [2/10], Loss: 0.6919488906860352
Epoch [3/10], Loss: 0.6964294910430908
Epoch [4/10], Loss: 0.6756751537322998
Epoch [5/10], Loss: 0.6902875304222107
Epoch [6/10], Loss: 0.6616805195808411
Epoch [7/10], Loss: 0.6635900735855103
Epoch [8/10], Loss: 0.6734824776649475
Epoch [9/10], Loss: 0.6482532620429993
Epoch [10/10], Loss: 0.644137978553772
show more (open the raw output data in a text editor) ...

Epoch [10/10], Loss: 0.644137978553772
checking accuracy on Training set
got 62 / 100 with accracy 62.0
checking accuracy on Test set
got 10 / 20 with accracy 50.0

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 64
num_epochs = 10

# transform 1

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    ToTensor()
])


# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Linear(5000, 2),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
#optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

    # training mode
    model.train()

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)

## Use resize when calc mean, std/ num_epoch = 10 / use transform 1/ data set num: 25000 / optim: SGD/ batch: 64

Epoch [1/10], Loss: 0.6884214878082275
Epoch [2/10], Loss: 0.6389276385307312
Epoch [3/10], Loss: 0.6553725600242615
Epoch [4/10], Loss: 0.6433557868003845
Epoch [5/10], Loss: 0.6392791271209717
Epoch [6/10], Loss: 0.659751296043396
Epoch [7/10], Loss: 0.6316360235214233
Epoch [8/10], Loss: 0.600490391254425
Epoch [9/10], Loss: 0.6885417699813843
Epoch [10/10], Loss: 0.6437572240829468
checking accuracy on Training set
got 13286 / 20000 with accracy 66.43
checking accuracy on Test set
got 3225 / 5000 with accracy 64.5

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 64
num_epochs = 10

# transform 1

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    ToTensor()
])


# Load data

dataset = CatAndDog('./cat_dog.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [20000, 5000]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Linear(5000, 2),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
#optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model).


















## Use randomresizedcrop when calc mean, std/ num_epoch = 10 / use transform 3/ data set num: 100 / optim: SGD/ batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer

Epoch [1/10], Loss: 0.6595055460929871
Epoch [2/10], Loss: 0.581857442855835
Epoch [3/10], Loss: 0.6393834948539734
Epoch [4/10], Loss: 0.8630320429801941
Epoch [5/10], Loss: 0.6885052919387817
Epoch [6/10], Loss: 0.5494183897972107
Epoch [7/10], Loss: 0.5923810601234436
Epoch [8/10], Loss: 0.6690357327461243
Epoch [9/10], Loss: 0.7105026245117188
Epoch [10/10], Loss: 0.6995693445205688
checking accuracy on Training set
got 54 / 100 with accracy 54.0
checking accuracy on Test set
got 9 / 20 with accracy 45.0

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
# from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

# transform 3
transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.4875, 0.4498, 0.4112], [0.2132, 0.2091, 0.2080], inplace=False),
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



## Use randomresizedcrop when calc mean, std/ num_epoch = 10 / use transform 2/ data set num: 100 / optim: SGD/ batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer

Epoch [1/10], Loss: 0.7946253418922424
Epoch [2/10], Loss: 0.6217297911643982
Epoch [3/10], Loss: 0.6169763803482056
Epoch [4/10], Loss: 0.5657680034637451
Epoch [5/10], Loss: 0.514208972454071
Epoch [6/10], Loss: 0.6392225623130798
Epoch [7/10], Loss: 0.41998255252838135
Epoch [8/10], Loss: 0.5450870990753174
Epoch [9/10], Loss: 0.41531461477279663
Epoch [10/10], Loss: 0.37717658281326294
checking accuracy on Training set
got 97 / 100 with accracy 97.0
checking accuracy on Test set
got 10 / 20 with accracy 50.0

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std)
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



## Use resize when calc mean, std/ num_epoch = 10 / use transform 2/ data set num: 100 / optim: SGD/ batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer

Epoch [1/10], Loss: 0.7497851848602295
Epoch [2/10], Loss: 0.6210096478462219
Epoch [3/10], Loss: 0.5349128246307373
Epoch [4/10], Loss: 0.5577841401100159
Epoch [5/10], Loss: 0.7867729067802429
Epoch [6/10], Loss: 0.8586550354957581
Epoch [7/10], Loss: 0.44590994715690613
Epoch [8/10], Loss: 0.4820903241634369
Epoch [9/10], Loss: 0.3990113139152527
Epoch [10/10], Loss: 0.27303779125213623
checking accuracy on Training set
got 99 / 100 with accracy 99.0
checking accuracy on Test set
got 9 / 20 with accracy 45.0

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std)
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



## Use resize when calc mean, std/ num_epoch = 10 / use transform 2/ data set num: 100 / optim: SGD/ batch: 32 / add sigmoid/ dropout location: before input layer, between linear, before output layer

Epoch [1/10], Loss: 0.6924512982368469
Epoch [2/10], Loss: 0.6731902360916138
Epoch [3/10], Loss: 0.7045927047729492
Epoch [4/10], Loss: 0.6608633399009705
Epoch [5/10], Loss: 0.7139399647712708
Epoch [6/10], Loss: 0.6770305633544922
Epoch [7/10], Loss: 0.6662049293518066
Epoch [8/10], Loss: 0.7075626850128174
Epoch [9/10], Loss: 0.6222336292266846
Epoch [10/10], Loss: 0.6222627758979797
checking accuracy on Training set
got 76 / 100 with accracy 76.0
checking accuracy on Test set
got 7 / 20 with accracy 35.0

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std)
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



## Use resize when calc mean, std/ num_epoch = 10 / use transform 2/ data set num: 25000 / optim: SGD/ batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std)
])

# Load data

dataset = CatAndDog('./cat_dog.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [20000, 5000]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



## Use resize when calc mean, std/ num_epoch = 30 / use transform + centercrop / data set num: 100 / optim: SGD/ batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer

Epoch [1/30], Loss: 0.661065936088562
Epoch [2/30], Loss: 0.4749096632003784
Epoch [3/30], Loss: 0.5973185896873474
Epoch [4/30], Loss: 0.6510248184204102
Epoch [5/30], Loss: 0.5774214863777161
Epoch [6/30], Loss: 0.5248855352401733
Epoch [7/30], Loss: 0.7273577451705933
Epoch [8/30], Loss: 0.5403749942779541
Epoch [9/30], Loss: 0.5342727899551392
Epoch [10/30], Loss: 0.4214780330657959
Epoch [11/30], Loss: 0.37489351630210876
Epoch [12/30], Loss: 0.3546043336391449
Epoch [13/30], Loss: 0.38444843888282776
Epoch [14/30], Loss: 0.5043830871582031
Epoch [15/30], Loss: 0.3135039210319519
Epoch [16/30], Loss: 0.3938744068145752
Epoch [17/30], Loss: 0.4075440466403961
Epoch [18/30], Loss: 0.315226674079895
Epoch [19/30], Loss: 0.23798835277557373
Epoch [20/30], Loss: 0.3003026843070984
Epoch [21/30], Loss: 0.25553005933761597
show more (open the raw output data in a text editor) ...

Epoch [30/30], Loss: 0.25391849875450134
checking accuracy on Training set
got 100 / 100 with accracy 100.0
checking accuracy on Test set
got 9 / 20 with accracy 45.0

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 30

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   transforms.CenterCrop(224),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std)
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



## Use resize when calc mean, std/ num_epoch = 30 / use transform + centercrop / data set num: 100 / optim: SGD + momentum/ batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer

Epoch [1/30], Loss: 0.6842379570007324
Epoch [2/30], Loss: 1.346143364906311
Epoch [3/30], Loss: 0.6354720592498779
Epoch [4/30], Loss: 0.42943471670150757
Epoch [5/30], Loss: 0.22744254767894745
Epoch [6/30], Loss: 0.1614459753036499
Epoch [7/30], Loss: 0.164921835064888
Epoch [8/30], Loss: 0.5951780676841736
Epoch [9/30], Loss: 0.14554078876972198
Epoch [10/30], Loss: 0.23797444999217987
Epoch [11/30], Loss: 0.15400224924087524
Epoch [12/30], Loss: 0.15590843558311462
Epoch [13/30], Loss: 0.13699601590633392
Epoch [14/30], Loss: 0.13801392912864685
Epoch [15/30], Loss: 0.1653336137533188
Epoch [16/30], Loss: 0.13161039352416992
Epoch [17/30], Loss: 0.16372820734977722
Epoch [18/30], Loss: 0.15361419320106506
Epoch [19/30], Loss: 0.1433015912771225
Epoch [20/30], Loss: 0.13895335793495178
Epoch [21/30], Loss: 0.1662384569644928
show more (open the raw output data in a text editor) ...

Epoch [30/30], Loss: 0.4052552580833435
checking accuracy on Training set
got 93 / 100 with accracy 93.0
checking accuracy on Test set
got 8 / 20 with accracy 40.0

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 30

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   transforms.CenterCrop(224),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std)
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



## Use resize when calc mean, std/ num_epoch = 20 / use transform + centercrop / data set num: 100 / optim: SGD / batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer

Epoch [1/20], Loss: 0.7280433177947998
Epoch [2/20], Loss: 0.6530508399009705
Epoch [3/20], Loss: 0.5301629304885864
Epoch [4/20], Loss: 0.5787199139595032
Epoch [5/20], Loss: 0.5401037931442261
Epoch [6/20], Loss: 0.5569936037063599
Epoch [7/20], Loss: 0.49162575602531433
Epoch [8/20], Loss: 0.38387924432754517
Epoch [9/20], Loss: 0.44622543454170227
Epoch [10/20], Loss: 0.26795244216918945
Epoch [11/20], Loss: 0.4028141498565674
Epoch [12/20], Loss: 0.36601412296295166
Epoch [13/20], Loss: 0.30876222252845764
Epoch [14/20], Loss: 0.3530395030975342
Epoch [15/20], Loss: 0.34304285049438477
Epoch [16/20], Loss: 0.4777611792087555
Epoch [17/20], Loss: 0.3471497893333435
Epoch [18/20], Loss: 0.2962343990802765
Epoch [19/20], Loss: 0.28462713956832886
Epoch [20/20], Loss: 0.2860521674156189
checking accuracy on Training set
got 99 / 100 with accracy 99.0
checking accuracy on Test set
got 8 / 20 with accracy 40.0

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 20

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   transforms.CenterCrop(224),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std)
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



## Use resize when calc mean, std/ num_epoch = 20 / use transform + centercrop / data set num: 100 / optim: SGD / batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer + model.train(), model.eval()

Epoch [1/20], Train Loss: 0.7650482654571533, Test Loss: 0.758185088634491, Test Accuracy: 55.0%
Epoch [2/20], Train Loss: 0.7518641948699951, Test Loss: 0.7249711155891418, Test Accuracy: 65.0%
Epoch [3/20], Train Loss: 0.5475682616233826, Test Loss: 0.7474795579910278, Test Accuracy: 55.0%
Epoch [4/20], Train Loss: 0.470769464969635, Test Loss: 0.7448733448982239, Test Accuracy: 45.0%
Epoch [5/20], Train Loss: 0.552834153175354, Test Loss: 0.7176886200904846, Test Accuracy: 60.000003814697266%
Epoch [6/20], Train Loss: 0.663266122341156, Test Loss: 0.7068558931350708, Test Accuracy: 45.0%
Epoch [7/20], Train Loss: 0.49595630168914795, Test Loss: 0.7391536831855774, Test Accuracy: 50.0%
Epoch [8/20], Train Loss: 0.4366358816623688, Test Loss: 0.7618598341941833, Test Accuracy: 50.0%
Epoch [9/20], Train Loss: 0.46581751108169556, Test Loss: 0.7799816131591797, Test Accuracy: 50.0%
Epoch [10/20], Train Loss: 0.3558882772922516, Test Loss: 0.7590056657791138, Test Accuracy: 45.0%
Epoch [11/20], Train Loss: 0.3713186979293823, Test Loss: 0.7488439083099365, Test Accuracy: 55.0%
Epoch [12/20], Train Loss: 0.3107163906097412, Test Loss: 0.7626556158065796, Test Accuracy: 55.0%
Epoch [13/20], Train Loss: 0.40933090448379517, Test Loss: 0.7605164647102356, Test Accuracy: 55.0%
Epoch [14/20], Train Loss: 0.3455810546875, Test Loss: 0.7621035575866699, Test Accuracy: 50.0%
Epoch [15/20], Train Loss: 0.3832550644874573, Test Loss: 0.7801468968391418, Test Accuracy: 50.0%
Epoch [16/20], Train Loss: 0.3303754925727844, Test Loss: 0.7671874165534973, Test Accuracy: 50.0%
Epoch [17/20], Train Loss: 0.3633565306663513, Test Loss: 0.7718254923820496, Test Accuracy: 50.0%
Epoch [18/20], Train Loss: 0.34651342034339905, Test Loss: 0.7692532539367676, Test Accuracy: 50.0%
Epoch [19/20], Train Loss: 0.2730775475502014, Test Loss: 0.7790290117263794, Test Accuracy: 50.0%
Epoch [20/20], Train Loss: 0.3114137053489685, Test Loss: 0.7907519340515137, Test Accuracy: 50.0%

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 20

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   transforms.CenterCrop(224),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std)
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.flatten(x)
        #x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training
def train_model(model, train_loader, optimizer, criterion, device):
    model.train()  

    for images, labels in train_loader:
        images = images.to(device)  
        labels = labels.to(device)  

        # forward
        class_scores = model(images)
        loss = criterion(class_scores, labels)

        # backward for calculating gradients and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return loss.item()

def test_model(model, test_loader, criterion, device):
    model.eval()  

    total_loss = 0.0
    num_correct = 0
    num_samples = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)  
            labels = labels.to(device)  

            # forward
            class_scores = model(images)
            loss = criterion(class_scores, labels)

            total_loss += loss.item()

            predictions = class_scores.argmax(dim=1)
            num_correct += (predictions == labels.argmax(1)).sum()
            num_samples += predictions.size(0)

    average_loss = total_loss / len(test_loader)
    accuracy = (num_correct / num_samples) * 100.0

    return average_loss, accuracy

# training and testing loop
for epoch in range(num_epochs):
    train_loss = train_model(model, train_loader, optimizer, criterion, device)
    test_loss, test_accuracy = test_model(model, test_loader, criterion, device)

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss}, Test Loss: {test_loss}, Test Accuracy: {test_accuracy}%')




## Use resize when calc mean, std/ num_epoch = 30 / use transform + centercrop / data set num: 100 / optim: SGD / batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer + model.train(), model.eval() 

Epoch [1/30], Loss: 0.5516689419746399
Epoch [2/30], Loss: 0.6553864479064941
Epoch [3/30], Loss: 0.5157697796821594
Epoch [4/30], Loss: 0.6209410429000854
Epoch [5/30], Loss: 0.6378206014633179
Epoch [6/30], Loss: 0.5514135956764221
Epoch [7/30], Loss: 0.9291189908981323
Epoch [8/30], Loss: 0.49721235036849976
Epoch [9/30], Loss: 0.7463662624359131
Epoch [10/30], Loss: 0.6215643286705017
Epoch [11/30], Loss: 0.6313633322715759     
Epoch [12/30], Loss: 0.5122519135475159
Epoch [13/30], Loss: 0.4463738203048706
Epoch [14/30], Loss: 0.5038759112358093
Epoch [15/30], Loss: 0.7185090780258179
Epoch [16/30], Loss: 0.45670998096466064
Epoch [17/30], Loss: 0.4213845133781433
Epoch [18/30], Loss: 0.3648272454738617
Epoch [19/30], Loss: 0.44255468249320984
Epoch [20/30], Loss: 0.5658482909202576
Epoch [21/30], Loss: 0.604422390460968
show more (open the raw output data in a text editor) ...

Epoch [30/30], Loss: 0.2924915850162506
checking accuracy on Training set
got 99 / 100 with accracy 99.0
checking accuracy on Test set
got 13 / 20 with accracy 65.0

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
#learning_rate = 1e-3
learning_rate = 1e-4
batch_size = 32
num_epochs = 30

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   transforms.CenterCrop(224),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std)
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            #nn.Tanh(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    model.train()

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



## Use resize when calc mean, std/ num_epoch = 30 / use transform + centercrop / data set num: 100 / optim: SGD / batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer + model.train(), model.eval() / batch normalization

Epoch [1/30], Loss: 1.0136306285858154
Epoch [2/30], Loss: 0.5256384611129761
Epoch [3/30], Loss: 0.5304974913597107
Epoch [4/30], Loss: 0.6144054532051086
Epoch [5/30], Loss: 0.656862199306488
Epoch [6/30], Loss: 0.8965831398963928
Epoch [7/30], Loss: 0.6497758030891418
Epoch [8/30], Loss: 0.7635592222213745
Epoch [9/30], Loss: 0.534446120262146
Epoch [10/30], Loss: 0.5434191226959229
Epoch [11/30], Loss: 0.5402575731277466
Epoch [12/30], Loss: 0.5119021534919739
Epoch [13/30], Loss: 0.45309364795684814
Epoch [14/30], Loss: 0.4040907025337219
Epoch [15/30], Loss: 0.35334813594818115
Epoch [16/30], Loss: 0.25867030024528503
Epoch [17/30], Loss: 0.25990626215934753
Epoch [18/30], Loss: 0.45160946249961853
Epoch [19/30], Loss: 0.34137481451034546
Epoch [20/30], Loss: 0.3909694254398346
Epoch [21/30], Loss: 0.5323057174682617
show more (open the raw output data in a text editor) ...

Epoch [30/30], Loss: 0.5978360176086426
checking accuracy on Training set
got 100 / 100 with accracy 100.0
checking accuracy on Test set
got 10 / 20 with accracy 50.0

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 30

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   transforms.CenterCrop(224),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std)
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.BatchNorm1d(10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.BatchNorm1d(5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    model.train()

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



## Use resize when calc mean, std/ num_epoch = 17 / use transform + centercrop / data set num: 100 / optim: SGD / batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer + model.train(), model.eval() / batch normalization

Epoch [1/17], Loss: 0.9410517811775208
Epoch [2/17], Loss: 0.7829709053039551
Epoch [3/17], Loss: 0.7021951675415039
Epoch [4/17], Loss: 0.7969115972518921
Epoch [5/17], Loss: 0.34849706292152405
Epoch [6/17], Loss: 0.7657426595687866
Epoch [7/17], Loss: 0.836965799331665
Epoch [8/17], Loss: 0.44204747676849365
Epoch [9/17], Loss: 1.0368456840515137
Epoch [10/17], Loss: 0.653814435005188
Epoch [11/17], Loss: 0.17304733395576477
Epoch [12/17], Loss: 0.8492071032524109
Epoch [13/17], Loss: 0.3307250738143921
Epoch [14/17], Loss: 0.33910369873046875
Epoch [15/17], Loss: 0.28491243720054626
Epoch [16/17], Loss: 0.34037286043167114
Epoch [17/17], Loss: 0.7725914120674133
checking accuracy on Training set
got 99 / 100 with accracy 99.0
checking accuracy on Test set
got 8 / 20 with accracy 40.0

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 17

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   transforms.CenterCrop(224),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std)
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.BatchNorm1d(10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.BatchNorm1d(5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    model.train()

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



## Use resize when calc mean, std/ num_epoch = 30 / use transform + centercrop + colorJitter / data set num: 100 / optim: SGD / batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer + model.train(), model.eval() 

Epoch [1/30], Loss: 0.7230271100997925
Epoch [2/30], Loss: 0.7384036779403687
Epoch [3/30], Loss: 0.7491973042488098
Epoch [4/30], Loss: 0.8197749853134155
Epoch [5/30], Loss: 0.6364127993583679
Epoch [6/30], Loss: 0.6486576199531555
Epoch [7/30], Loss: 0.6163468360900879
Epoch [8/30], Loss: 0.8640875816345215
Epoch [9/30], Loss: 0.6495388150215149
Epoch [10/30], Loss: 0.6619004011154175
Epoch [11/30], Loss: 0.642125129699707
Epoch [12/30], Loss: 0.5758170485496521
Epoch [13/30], Loss: 0.6188790798187256
Epoch [14/30], Loss: 0.75055992603302
Epoch [15/30], Loss: 0.5563116669654846
Epoch [16/30], Loss: 0.6785495281219482
Epoch [17/30], Loss: 0.6502804756164551
Epoch [18/30], Loss: 0.6345024704933167
Epoch [19/30], Loss: 0.5741427540779114
Epoch [20/30], Loss: 0.5574901700019836
Epoch [21/30], Loss: 0.5354763865470886
show more (open the raw output data in a text editor) ...

Epoch [30/30], Loss: 0.6613378524780273
checking accuracy on Training set
got 92 / 100 with accracy 92.0
checking accuracy on Test set
got 12 / 20 with accracy 60.0

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 30

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   transforms.CenterCrop(224),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    model.train()

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



## Use resize when calc mean, std/ num_epoch = 30 / use transform + centercrop / data set num: 100 / optim: SGD + momentum / batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer + model.train(), model.eval() 

Epoch [1/30], Loss: 0.681688129901886
Epoch [2/30], Loss: 0.5502901673316956
Epoch [3/30], Loss: 0.5514283180236816
Epoch [4/30], Loss: 0.6526256203651428
Epoch [5/30], Loss: 0.3460688591003418
Epoch [6/30], Loss: 0.37935906648635864
Epoch [7/30], Loss: 0.5031934976577759
Epoch [8/30], Loss: 0.4837206304073334
Epoch [9/30], Loss: 0.45266300439834595
Epoch [10/30], Loss: 0.19656500220298767
Epoch [11/30], Loss: 0.176497682929039
Epoch [12/30], Loss: 0.1445126086473465
Epoch [13/30], Loss: 0.22931212186813354
Epoch [14/30], Loss: 0.13587144017219543
Epoch [15/30], Loss: 0.16981680691242218
Epoch [16/30], Loss: 0.15087677538394928
Epoch [17/30], Loss: 0.13050831854343414
Epoch [18/30], Loss: 0.1320030689239502
Epoch [19/30], Loss: 0.3189740478992462
Epoch [20/30], Loss: 0.12889617681503296
Epoch [21/30], Loss: 0.1521141231060028
show more (open the raw output data in a text editor) ...

Epoch [30/30], Loss: 0.13535752892494202
checking accuracy on Training set
got 96 / 100 with accracy 96.0
checking accuracy on Test set
got 10 / 20 with accracy 50.0

In [ ]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-3
batch_size = 32
num_epochs = 30

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   transforms.CenterCrop(224),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std)
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        #drop out
        self.dropout = nn.Dropout(p=0.5)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3 * 224 * 224, 10000),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(10000, 5000),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(5000, 2),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.dropout(x)
        logits = self.linear_relu_stack(x)

        # logit = class score tensor
        # print('checking Forward_NeuralNetwork')
        return logits

# model

model = NeuralNetwork().to(device)
# print(model)

# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    model.train()

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



## (CNN) Use resize when calc mean, std/ num_epoch = 30 / use transform + centercrop / data set num: 100 / optim: SGD / batch: 32 / add tanh/ dropout location: before input layer, between linear, before output layer + model.train(), model.eval() 

Epoch [1/30], Loss: 0.6998062133789062
Epoch [2/30], Loss: 0.6833628416061401
Epoch [3/30], Loss: 0.6833722591400146
Epoch [4/30], Loss: 0.7176940441131592
Epoch [5/30], Loss: 0.6986485123634338
Epoch [6/30], Loss: 0.6862339973449707
Epoch [7/30], Loss: 0.6943838596343994
Epoch [8/30], Loss: 0.6894701719284058
Epoch [9/30], Loss: 0.6983991861343384
Epoch [10/30], Loss: 0.7059083580970764
Epoch [11/30], Loss: 0.6926674842834473
Epoch [12/30], Loss: 0.6815227270126343
Epoch [13/30], Loss: 0.6734297871589661
Epoch [14/30], Loss: 0.6712950468063354
Epoch [15/30], Loss: 0.683769941329956
Epoch [16/30], Loss: 0.693181037902832
Epoch [17/30], Loss: 0.6790727972984314
Epoch [18/30], Loss: 0.6740067005157471
Epoch [19/30], Loss: 0.6791797876358032
Epoch [20/30], Loss: 0.6501713991165161
show more (open the raw output data in a text editor) ...

Epoch [30/30], Loss: 0.6508783102035522
checking accuracy on Training set
got 78 / 100 with accracy 78.0
checking accuracy on Test set
got 10 / 20 with accracy 50.0

In [1]:
import import_ipynb
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader
from cat_and_dog_dataset import CatAndDog
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from img_normal import mean, std
import torch.nn.functional as F

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
# print(f"Using {device} device")

# Hyperparam

num_classes = 2
learning_rate = 1e-4
batch_size = 32
num_epochs = 30

# transform 2
transform = transforms.Compose([
   transforms.Resize((224, 224)),
   transforms.CenterCrop(224),
   ToTensor(),
#    transforms.Normalize([0.4883, 0.4551, 0.4170], [0.2256, 0.2211, 0.2213], inplace=False)
    transforms.Normalize(mean, std)
])

# Load data

dataset = CatAndDog('./cat_dog_100.csv', 
                    './cat_dog/image/', 
                    # transform = ToTensor(), 
                    # target_transform=Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
                    )
dataset.transform = transform
# dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
dataset.target_transform = Lambda(lambda y: torch.zeros(num_classes, dtype=torch.float).scatter_(0, y.clone().detach(), value=1))


# train set = 80%, test_set = 20%
train_set, test_set = torch.utils.data.random_split(dataset, [100, 20]) 

train_loader = DataLoader(dataset = train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set, batch_size=batch_size, shuffle=True)


# print("Checking dataset...")


# for i in range(5):
#     image, label = dataset[i]
#     print(f"Image {i} shape: {image.shape}, Label: {label}")

# print("\nChecking train loader...")


# for images, labels in train_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  

# print("\nChecking test loader...")


# for images, labels in test_loader:
#     print(f"Batch images shape: {images.shape}, Batch labels shape: {labels.shape}")
#     break  


# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.flatten = nn.Flatten()

#         #drop out
#         self.dropout = nn.Dropout(p=0.5)
#         self.linear_relu_stack = nn.Sequential(
#             nn.Linear(3 * 224 * 224, 10000),
#             nn.ReLU(),
#             nn.Dropout(p=0.5),

#             nn.Linear(10000, 5000),
#             nn.ReLU(),
#             nn.Dropout(p=0.5),
            
#             nn.Linear(5000, 2),
#             nn.Tanh(),
#         )

#     def forward(self, x):
#         x = self.flatten(x)
#         x = self.dropout(x)
#         logits = self.linear_relu_stack(x)

#         # logit = class score tensor
#         # print('checking Forward_NeuralNetwork')
#         return logits

class CNN(nn.Module):
    def __init__(self, n_output, n_hidden):
        super().__init__()

        self.flatten = nn.Flatten()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)

        # Max pooling layers
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Fully connected layers (224 * 224 -> 112 * 112 -> 56 * 56 -> 28 * 28)
        self.fc1 = nn.Linear(128 * 28 * 28, 1000)
        self.fc2 = nn.Linear(1000, n_output)

        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        x = self.fc2(x)

        return x


# model
model = CNN(n_output=num_classes, n_hidden=1000).to(device)



# calc loss 
#criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()


# Update the weight
optimizer = optim.SGD(model.parameters(), lr=learning_rate) 
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) 

# training model
for epoch in range(num_epochs):

    model.train()

    # update and store into device
    for images, labels in train_loader:
        images = images.to(device)  
        label = labels.to(device)  

        # forward
        class_score = model(images)
        loss = criterion(class_score, labels.float())

        # bakcward for calculating gradient and update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    # test
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    
# check accuracy
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    # evaluation mode
    model.eval()

    # forward to check accuracy
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)

            score = model(x)

            prediction = score.argmax(1)
            num_correct += (prediction == y.argmax(1)).sum()
            num_samples += prediction.size(0)

        print(f'got {num_correct} / {num_samples} with accracy {float(num_correct)/float(num_samples) * 100}')

            

# test

print('checking accuracy on Training set')
check_accuracy(train_loader, model)

print('checking accuracy on Test set')
check_accuracy(test_loader, model)



importing Jupyter notebook from cat_and_dog_dataset.ipynb
Dataset size: 120
importing Jupyter notebook from img_normal.ipynb
Mean: tensor([0.4883, 0.4551, 0.4170])
Standard Deviation: tensor([0.2256, 0.2211, 0.2213])
Epoch [1/30], Loss: 0.6998062133789062
Epoch [2/30], Loss: 0.6833628416061401
Epoch [3/30], Loss: 0.6833722591400146
Epoch [4/30], Loss: 0.7176940441131592
Epoch [5/30], Loss: 0.6986485123634338
Epoch [6/30], Loss: 0.6862339973449707
Epoch [7/30], Loss: 0.6943838596343994
Epoch [8/30], Loss: 0.6894701719284058
Epoch [9/30], Loss: 0.6983991861343384
Epoch [10/30], Loss: 0.7059083580970764
Epoch [11/30], Loss: 0.6926674842834473
Epoch [12/30], Loss: 0.6815227270126343
Epoch [13/30], Loss: 0.6734297871589661
Epoch [14/30], Loss: 0.6712950468063354
Epoch [15/30], Loss: 0.683769941329956
Epoch [16/30], Loss: 0.693181037902832
Epoch [17/30], Loss: 0.6790727972984314
Epoch [18/30], Loss: 0.6740067005157471
Epoch [19/30], Loss: 0.6791797876358032
Epoch [20/30], Loss: 0.65017139911